In [1]:
import os
import sys
import time
from naoqi import ALProxy
import numpy as np
import cv2
from datetime import datetime

In [2]:
NAO_IP = "10.0.107.217"
NAO_PORT = 9559
PEPPER_IP = "10.0.52.247"
PEPPER_PORT = 9503

In [ ]:
def increase_brightness(img, value=80):
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  h, s, v = cv2.split(hsv)

  lim = 255 - value
  v[v > lim] = 255
  v[v <= lim] += value

  final_hsv = cv2.merge((h, s, v))
  img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
  return img

def get_feed(tts,subscriberID):
  nao_image = tts.getImageRemote(str(subscriberID))

  img = (np.reshape(np.frombuffer(nao_image[6], dtype = '%iuint8' % nao_image[2]), (nao_image[1], nao_image[0], nao_image[2])))
  img = np.array(img)
  
  #img = increase_brightness(img)

  # if(i == 0):  
  frame_width = int(nao_image[0])
  frame_height = int(nao_image[1])
  size = (frame_width, frame_height)
  result = cv2.VideoWriter('filename.avi', -1 , 10, size)
    # i += 1
  # define the alpha and beta
  #contrast = 5. # Contrast control ( 0 to 127)
  #brightness = 70. # Brightness control (0-100)

  # call convertScaleAbs function
  #img = cv2.addWeighted( img, contrast, img, 0, brightness)

  # info = result.write(img)
  result.write(img)
  # print(info)
  cv2.imshow("Input", img)
  # if (i > 100):
  #   stop = True
  # time.sleep(5)
  k = cv2.waitKey(33)
  # if k==27:    # Esc key to stop
  #   break
  return result

Video capture

In [ ]:
tts = ALProxy("ALVideoDevice", NAO_IP, NAO_PORT)
camera_index = 0
resolution = 3 # for VGA (640 x 480)
colourspace = 11
FPS = 5
subscriberID = tts.subscribeCamera("subscriberID", camera_index, resolution,colourspace, FPS)
tts.openCamera(camera_index)
tts.startCamera(camera_index)

faceProxy = ALProxy("ALFaceDetection", NAO_IP, NAO_PORT)

print("ID is :", subscriberID)

ref_time = datetime.now()
present_time = datetime.now()
diff_in_time = 0 
while (diff_in_time != 5):
  # result = get_feed(tts,subscriberID)
  # Implement the logic here 

  nao_image = tts.getImageRemote(str(subscriberID))

  img = (np.reshape(np.frombuffer(nao_image[6], dtype = '%iuint8' % nao_image[2]), (nao_image[1], nao_image[0], nao_image[2])))
  img = np.array(img)

  frame_width = int(nao_image[0])
  frame_height = int(nao_image[1])
  size = (frame_width, frame_height)
  result = cv2.VideoWriter('filename.avi', -1 , 10, size)

  result.write(img)
  # print(info)
  cv2.imshow("Input", img)
  
  # End of logic
  present_time = datetime.now()
  diff_in_time = int((present_time - ref_time).total_seconds())
  # print("The time is :- ", diff_in_time)
  k = cv2.waitKey(33)
  if k == 27: # Esc to stop 
    break
  # print ("The diff in the time is:-",diff_in_time)
result.release()
tts.releaseImage("subscriberID_5")
tts.unsubscribe("subscriberID_5")
cv2.destroyAllWindows()

Image Capture

In [4]:
# Create a proxy to ALPhotoCapture
try:
  photoCaptureProxy = ALProxy("ALPhotoCapture", NAO_IP, NAO_PORT)
except Exception, e:
  print "Error when creating ALPhotoCapture proxy:"
  print str(e)
  exit(1)

# Take 3 pictures in VGA and store them in /home/nao/recordings/cameras/

photoCaptureProxy.setResolution(2)
photoCaptureProxy.setPictureFormat("jpg")
photoCaptureProxy.takePictures(3, ".", "image")

# This call returns ['/home/nao/recordings/cameras/image_0.jpg', '/home/nao/recordings/cameras/image_1.jpg', '/home/nao/recordings/cameras/image_2.jpg']

RuntimeError: 	ALPhotoCapture::takePictures
		ALPhotoCapture::takePictures
	The path /home/sougato97/Human_Robot_Interaction cannot be created